In [0]:
%sql
USE CATALOG agriculture

In [0]:
%sql
-- Create the Final ROI Table with "Simulated Time" for the Demo
CREATE OR REPLACE TABLE agriculture.gold.roi_recommendations AS
WITH Yield_Benchmarks AS (
    SELECT 
        state,
        district,
        crop,
        AVG(calculated_yield) as avg_yield_per_hectare
    FROM agriculture.silver.crop_production
    GROUP BY state, district, crop
),
Latest_Forecast AS (
    SELECT 
        crop_name,
        -- Shift dates by 24 months to simulate "Current" forecast
        -- Original forecast was 2024. We make it 2026.
        add_months(ds, 24) as forecast_date, 
        yhat as predicted_price,
        price_stability_score as risk_factor
    FROM agriculture.gold.market_forecasts
)
SELECT 
    y.state,
    y.district,
    y.crop as crop_name,
    f.forecast_date,
    y.avg_yield_per_hectare,
    f.predicted_price,
    f.risk_factor,
    (y.avg_yield_per_hectare * f.predicted_price) as estimated_revenue,
    CASE 
        WHEN f.risk_factor < 0.1 THEN 'Low Risk'
        WHEN f.risk_factor < 0.2 THEN 'Medium Risk'
        ELSE 'High Risk (Volatile Market)'
    END as risk_category
FROM Yield_Benchmarks y
JOIN Latest_Forecast f ON lower(y.crop) = lower(f.crop_name)
-- Filter to show only "Future" predictions relative to "Today" (simulated)
WHERE f.forecast_date >= current_date();



In [0]:
%sql
-- Check the "Time Traveled" results
SELECT * FROM agriculture.gold.roi_recommendations ORDER BY forecast_date ASC LIMIT 10;